In [7]:

file_path = './dataset/dataset2.csv'

with open (file_path, 'r', encoding='latin1') as file:
    text = file.read()

###     
### carico il dataset dal file originale e faccio un processo di encoding e decoding da utf-8 per eliminare i caratteri speciali
###
text = text.replace('\x92', "'")
with open('./dataset/editato.csv', 'w', encoding='utf-8') as file2:
    file2.write(text.encode(encoding='utf-8').decode())

In [8]:
import pandas as pd

df = pd.read_csv('./dataset/editato.csv', encoding='utf-8')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259569 entries, 0 to 259568
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   tweet_date_created  259569 non-null  object
 1   tweet_id            259569 non-null  int64 
 2   tweet_text          259569 non-null  object
 3   language            259569 non-null  object
 4   sentiment           259569 non-null  object
 5   sentiment_score     259569 non-null  object
dtypes: int64(1), object(5)
memory usage: 11.9+ MB


In [9]:
sentiment_counts = df['sentiment'].value_counts()

# Stampa i risultati
print("Numero di elementi con sentimento Neutral più alto:", sentiment_counts['NEUTRAL'])
print("Numero di elementi con sentimento Negative più alto:", sentiment_counts['NEGATIVE'])
print("Numero di elementi con sentimento Positive più alto:", sentiment_counts['POSITIVE'])
print("Numero di elementi con sentimento Mixed più alto:", sentiment_counts['MIXED'])

Numero di elementi con sentimento Neutral più alto: 226868
Numero di elementi con sentimento Negative più alto: 9441
Numero di elementi con sentimento Positive più alto: 22883
Numero di elementi con sentimento Mixed più alto: 377


In [10]:
import re 

def rimuovi_mentions_hashtag_e_link(testo):
    return re.sub(r'(@[\w\d]+|#[\w\d]+|https://t\.co/[\w\d]+)', '', testo)

def rimuovi_punteggiatura(testo):
    return re.sub(r'[^\w\s]', ' ', testo.lower())


# Applica la funzione a ciascuna riga della colonna 'tweet_text'
df['tweet_text'] = df['tweet_text'].apply(rimuovi_mentions_hashtag_e_link)
df['tweet_text'] = df['tweet_text'].apply(rimuovi_punteggiatura)

In [ ]:
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


# nlp = spacy.load("it_core_news_sm")

stop_words = set(stopwords.words('italian'))
preposizioni_articolate = [
    "al", "allo", "alla", "ai", "agli", "alla",
    "del", "dello", "della", "dei", "degli", "delle",
    "nel", "nello", "nella", "nei", "negli", "nelle",
    "sul", "sullo", "sulla", "sui", "sugli", "sulle",
    "col", "collo", "colla", "coi", "cogli", "colle",
    "dal", "dallo", "dalla", "dai", "dagli", "dalle",
    "nel", "nello", "nella", "nei", "negli", "nelle",
    "pel", "pello", "pella", "pei", "pegli", "pelle",
    "sul", "sullo", "sulla", "sui", "sugli", "sulle",
    "con","d","c","l"
]
stop_words.update(preposizioni_articolate)

def tokenizza_testo(testo):
    return word_tokenize(testo)

def lemmatizza_testo(parole_tokenizzate):
    parole_lemmatizzate = []
    for token in nlp(" ".join(parole_tokenizzate)):
        if token.lemma_.lower() not in stop_words:
            parole_lemmatizzate.append(token.lemma_)
    return parole_lemmatizzate

def rimuovi_stop_words(parole):
    return [parola for parola in parole if parola.lower() not in stop_words]



def processa_testo(testo):
    parole_tokenizzate = tokenizza_testo(testo)
    parole_pulite = rimuovi_stop_words(parole_tokenizzate)
    parole_lemmatizzate = lemmatizza_testo(parole_pulite)
    return parole_lemmatizzate


df['processed_text'] = df['tweet_text'].apply(processa_testo)

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\giova\AppData\Local\Programs\Python\Python310\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\giova\AppData\Local\Programs\Python\Python310\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Connessione in corso interrotta forzatamente dall'host remoto
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\giova\AppData\Local\Programs\Python\Python310\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\giova\AppData\Local\Programs\Python\Python310\lib\asyncio\selector_events.py", line 115, in _read_from_se

In [ ]:
#salvataggio del dataset ottenuto dalla fase di preprocessing in un file csv
df.to_csv('./dataset/processed.csv', index=False)

In [ ]:
df = pd.read_csv('processed.csv')

df = df[['processed_text', 'sentiment']]

df['processed_text'] = df['processed_text'].apply(lambda x: x.replace('[','').replace(']','').replace(',','').replace("'",''))

df.to_csv('./dataset/testo_e_sentiment.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('testo_e_sentiment.csv')
num_negative = (df['sentiment'] == 'NEGATIVE').sum()

undersampled_df = pd.concat([
    df[df['sentiment'] == 'NEGATIVE'],  
    df[df['sentiment'] == 'POSITIVE'].sample(n=num_negative, random_state=42),  
    df[df['sentiment'] == 'NEUTRAL'].sample(n=num_negative, random_state=42) 
])

undersampled_df = undersampled_df.sample(frac=1, random_state=42)

print(undersampled_df['sentiment'].value_counts())

undersampled_df.to_csv('./dataset/undersampled.csv', index=False)